In [ ]:
import pandas as pd
from datetime import datetime
import pytz
from pytz import timezone
import re
import csv
import requests
import json
import time
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [8]:
#opening json file with stored credentials
with open('creds.json') as f:
    creds = json.load(f)

In [11]:
payload = {}
#loading in key values from the json needed to make the query
headers = {
      'Authorization': f"Bearer {creds['bearer_token']}",
      'Cookie': f"personalization_id={creds['personalization_id']}; guest_id={creds['guest_id']}"
    }

base_url = f"https://api.twitter.com/1.1/tweets/search/30day/dev.json?fromDate=202109040000&toDate=202110040000&query=point_radius:[-82.3666 23.1136 25mi] -is:retweet -is:reply OR profile_country:AF -is:reply -is:retweet profile_locality:Afghanistan
'''the query for the data, collecting all tweets 
for the month of August, within 25 miles of Kabul 
with a derived location of Afghanistan'''

count = 1
check=True
while check==True:
    if count == 1:
        url = base_url
#using the base url for the first request
    
    else:
        url = base_url + "&next={0}".format(next_page)
#combining the base url and the url extension for a new auery   
    
    response = requests.request("GET", url, headers=headers, data = payload)
#retrieving the query data
    
    jsonfile_name = f'data/data{count}.json'
    
    d = json.loads(response.text)
#loading in the json data
    
    if d['results']:
        with open(jsonfile_name, 'w') as f:
            json.dump(d, f)

#loading the flattened json data into the file
        next_page = d['next']

#saving the url extension for the next page, used to create the query in the next iteration
    elif d['error'] or len(d['next']) == 0:
        print(d)
        check==False
        break
        
    count += 1
    time.sleep(5)